In [1]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()

print(data)

{'endpoint': 'Historical Options', 'message': 'success', 'data': [{'contractID': 'IBM250627C00130000', 'symbol': 'IBM', 'expiration': '2025-06-27', 'strike': '130.00', 'type': 'call', 'last': '0.00', 'mark': '159.62', 'bid': '157.65', 'bid_size': '73', 'ask': '161.60', 'ask_size': '1', 'volume': '0', 'open_interest': '0', 'date': '2025-06-27', 'implied_volatility': '4.42386', 'delta': '1.00000', 'gamma': '0.00000', 'theta': '-0.01542', 'vega': '0.00000', 'rho': '0.00356'}, {'contractID': 'IBM250627P00130000', 'symbol': 'IBM', 'expiration': '2025-06-27', 'strike': '130.00', 'type': 'put', 'last': '0.00', 'mark': '0.01', 'bid': '0.00', 'bid_size': '0', 'ask': '1.27', 'ask_size': '1', 'volume': '0', 'open_interest': '0', 'date': '2025-06-27', 'implied_volatility': '4.82688', 'delta': '-0.00049', 'gamma': '0.00002', 'theta': '-0.06335', 'vega': '0.00026', 'rho': '-0.00000'}, {'contractID': 'IBM250627C00135000', 'symbol': 'IBM', 'expiration': '2025-06-27', 'strike': '135.00', 'type': 'call'

In [2]:
data

{'endpoint': 'Historical Options',
 'message': 'success',
 'data': [{'contractID': 'IBM250627C00130000',
   'symbol': 'IBM',
   'expiration': '2025-06-27',
   'strike': '130.00',
   'type': 'call',
   'last': '0.00',
   'mark': '159.62',
   'bid': '157.65',
   'bid_size': '73',
   'ask': '161.60',
   'ask_size': '1',
   'volume': '0',
   'open_interest': '0',
   'date': '2025-06-27',
   'implied_volatility': '4.42386',
   'delta': '1.00000',
   'gamma': '0.00000',
   'theta': '-0.01542',
   'vega': '0.00000',
   'rho': '0.00356'},
  {'contractID': 'IBM250627P00130000',
   'symbol': 'IBM',
   'expiration': '2025-06-27',
   'strike': '130.00',
   'type': 'put',
   'last': '0.00',
   'mark': '0.01',
   'bid': '0.00',
   'bid_size': '0',
   'ask': '1.27',
   'ask_size': '1',
   'volume': '0',
   'open_interest': '0',
   'date': '2025-06-27',
   'implied_volatility': '4.82688',
   'delta': '-0.00049',
   'gamma': '0.00002',
   'theta': '-0.06335',
   'vega': '0.00026',
   'rho': '-0.00000'}

In [5]:
import pandas as pd

# normalize the JSON data
df = pd.json_normalize(data['historicalOptions'])

KeyError: 'historicalOptions'

In [ ]:
def options_json_to_df(response_data, symbol=None, collection_date=None):
    """
    Convert Alpha Vantage options API response to a pandas DataFrame
    
    Parameters:
    - response_data: JSON response from Alpha Vantage
    - symbol: Optional symbol to fill in if missing
    - collection_date: Optional date when data was collected
    """
    if 'data' not in response_data or not response_data['data']:
        return pd.DataFrame()
    
    # Convert to DataFrame
    df = pd.DataFrame(response_data['data'])
    
    # Convert numeric columns
    numeric_cols = ['strike', 'last', 'mark', 'bid', 'ask', 
                   'implied_volatility', 'delta', 'gamma', 
                   'theta', 'vega', 'rho']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convert integer columns
    int_cols = ['bid_size', 'ask_size', 'volume', 'open_interest']
    for col in int_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')  # nullable integer type
    
    # Convert date columns
    date_cols = ['expiration', 'date']
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])
    
    # Fill in symbol if provided
    if symbol and 'symbol' in df.columns:
        df['symbol'].fillna(symbol, inplace=True)
    
    # Add collection date if provided
    if collection_date:
        df['collected_date'] = pd.to_datetime(collection_date)
    
    return df

# Test with our sample data
df = options_json_to_df(data, symbol='IBM')
print(f"Loaded {len(df)} option records")

In [9]:
df = pd.DataFrame(data['data'])


In [ ]:
# Example of how to use this in a data pipeline
def fetch_options_data(symbol, date, api_key):
    """Fetch options data for a symbol and date"""
    url = f'https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS&symbol={symbol}&date={date}&apikey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# Example usage (using demo key, replace with your key)
symbol = 'IBM'
date = '2025-06-30'
api_key = 'demo'  # Replace with your API key

# Fetch and convert to DataFrame in one go
data = fetch_options_data(symbol, date, api_key)
if data:
    df = options_json_to_df(data, symbol=symbol, collection_date=date)
    print(f"Shape: {df.shape}")
    print("\nColumn types:")
    print(df.dtypes)
    print("\nFirst few rows:")
    print(df.head())

In [10]:
df.columns

Index(['contractID', 'symbol', 'expiration', 'strike', 'type', 'last', 'mark',
       'bid', 'bid_size', 'ask', 'ask_size', 'volume', 'open_interest', 'date',
       'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'rho'],
      dtype='object')

In [11]:
df.shape

(1724, 20)

In [15]:
df['date'].unique()

array(['2025-06-27'], dtype=object)